# Interactive Commercial Dashboard (Notebook)

This notebook provides an **interactive dashboard** using **ipywidgets + IPython**.

It is designed to mirror the **Dash app** in `app/app.py`:
- Same filters (month range, segment, category, subcategory, Top N)
- Same KPIs and metric definitions
- Same tabs (Executive, Trends, Customers, Products, Data Quality)

If you prefer a web UI, run the Dash app instead.


**Dashboard location:** The interactive dashboard is generated in the **last cell** of this notebook.


In [31]:
"""Repository bootstrap for notebooks and ad-hoc execution.

This module ensures the repository root (the directory containing 'src/') is added
to sys.path so that imports like `from src...` work reliably when running
notebooks from the /notebooks folder (or any subfolder inside the repo).
"""
import bootstrap


In [32]:
# Notebook setup
#from __future__ import annotations

import sys
from pathlib import Path


import pandas as pd
import plotly.express as px

import ipywidgets as widgets
#import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from src.io import load_sample
from src.core import (
    money0,
    filter_df,
    kpis,
    compute_monthly,
    pareto_curve,
    dq_indicators,
)



In [33]:
# Load data (sample extract)
dfs = load_sample()
fact = dfs["fact_sales"]
dim_customers = dfs["report_customers"]
dim_products = dfs["report_products"]

# Ensure year_month exists for filtering
fact["order_date"] = pd.to_datetime(fact["order_date"], errors="coerce")
fact = fact.dropna(subset=["order_date"]).copy()
fact["year_month"] = fact["order_date"].dt.to_period("M").astype(str)

#fact_enriched = ensure_display_columns(fact)
fact_enriched = fact.merge(dim_customers, on="customer_key", how="left", suffixes=("", "_cust"))
fact_enriched = fact_enriched.merge(dim_products, on="product_key", how="left", suffixes=("", "_prd"))

months = sorted(fact_enriched["year_month"].dropna().unique().tolist())
min_month = months[0] if months else "—"
max_month = months[-2] if months else "—"

segments = ["All"] + (sorted([str(x) for x in fact_enriched["customer_segment"].dropna().unique()]) if "customer_segment" in fact_enriched.columns else [])
categories = ["All"] + (sorted([str(x) for x in fact_enriched["category"].dropna().unique()]) if "category" in fact_enriched.columns else [])
subcategories = ["All"] + (sorted([str(x) for x in fact_enriched["subcategory"].dropna().unique()]) if "subcategory" in fact_enriched.columns else [])

# Precompute sample-wide DQ indicators
dq = dq_indicators(fact_enriched, dim_customers, dim_products)

months[:3], min_month, max_month


(['2013-01', '2013-02', '2013-03'], '2013-01', '2013-12')

In [34]:
# Schema validation (fail fast if inputs change)
from src.core.schema_checks import assert_required_columns, REQUIRED

assert_required_columns(fact, REQUIRED["fact_sales"], "fact_sales")
assert_required_columns(dim_customers, REQUIRED["dim_customers"], "dim_customers")
assert_required_columns(dim_products, REQUIRED["dim_products"], "dim_products")

In [35]:
# Dashboard UI setup (run once)

start_dd = widgets.Dropdown(options=months, value=min_month, description="Start:", layout=widgets.Layout(width="220px"))
end_dd = widgets.Dropdown(options=months, value=max_month, description="End:", layout=widgets.Layout(width="220px"))
seg_dd = widgets.Dropdown(options=segments or ["All"], value="All", description="Segment:", layout=widgets.Layout(width="260px"))
cat_dd = widgets.Dropdown(options=categories or ["All"], value="All", description="Category:", layout=widgets.Layout(width="260px"))
subcat_dd = widgets.Dropdown(options=subcategories or ["All"], value="All", description="Subcat:", layout=widgets.Layout(width="260px"))
topn_sl = widgets.IntSlider(value=10, min=5, max=25, step=5, description="Top N:", continuous_update=False, layout=widgets.Layout(width="360px"))

filters = widgets.VBox([
    widgets.HBox([start_dd, end_dd, topn_sl]),
    widgets.HBox([seg_dd, cat_dd, subcat_dd]),
])

tabs = widgets.Tab()
tabs.children = (widgets.Output(), widgets.Output(), widgets.Output(), widgets.Output(), widgets.Output())
for i, name in enumerate(["Executive", "Trends", "Customers", "Products", "Data Quality"]):
    tabs.set_title(i, name)

dq = dq_indicators(fact_enriched, dim_customers, dim_products)

In [36]:
def render_all():
    global dq
    if 'dq' not in globals():
        dq = dq_indicators(fact_enriched, dim_customers, dim_products)

    s, e = start_dd.value, end_dd.value
    for out in tabs.children:
        out.clear_output()

    if s > e:
        with tabs.children[0]:
            display(widgets.HTML("<div style='color:crimson;'>Start month must be <= End month.</div>"))
        return

    df = filter_df(fact_enriched, s, e, seg_dd.value, cat_dd.value, subcat_dd.value)
    k = kpis(df)
    monthly = compute_monthly(df)

    # Executive
    with tabs.children[0]:
        display(widgets.HTML(
            f"""<div style='display:flex; gap:12px; flex-wrap:wrap;'>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>Revenue</div><div style='font-size:22px;font-weight:600;'>{money0(k['revenue'])}</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>Orders</div><div style='font-size:22px;font-weight:600;'>{k['orders']:,}</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>Units</div><div style='font-size:22px;font-weight:600;'>{k['units']:,.0f}</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>Active Customers</div><div style='font-size:22px;font-weight:600;'>{k['customers']:,}</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>AOV</div><div style='font-size:22px;font-weight:600;'>{money0(k['aov'])}</div>
                    <div style='font-size:12px;color:#777;'>Revenue / Orders</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>ASP</div><div style='font-size:22px;font-weight:600;'>{('$'+format(k['asp'], ',.2f')) if pd.notna(k['asp']) else '—'}</div>
                    <div style='font-size:12px;color:#777;'>Revenue / Units</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>Units / Order</div><div style='font-size:22px;font-weight:600;'>{(format(k['upo'], ',.2f')) if pd.notna(k['upo']) else '—'}</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>MoM Revenue</div><div style='font-size:22px;font-weight:600;'>{(format(k['latest_mom_revenue_pct'], '+.1f')+'%') if pd.notna(k['latest_mom_revenue_pct']) else '—'}</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>MoM Orders</div><div style='font-size:22px;font-weight:600;'>{(format(k['latest_mom_orders_pct'], '+.1f')+'%') if pd.notna(k['latest_mom_orders_pct']) else '—'}</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>Top 10 Cust Share</div><div style='font-size:22px;font-weight:600;'>{(format(k['top10_customer_share_pct'], '.1f')+'%') if pd.notna(k['top10_customer_share_pct']) else '—'}</div>
                </div>
                <div style='border:1px solid #ddd; border-radius:12px; padding:10px 14px; min-width:210px;'>
                    <div style='font-size:12px;color:#555;'>Top 10 Prod Share</div><div style='font-size:22px;font-weight:600;'>{(format(k['top10_product_share_pct'], '.1f')+'%') if pd.notna(k['top10_product_share_pct']) else '—'}</div>
                </div>
            </div>"""))

        fig = px.line(monthly, x="year_month", y="revenue", title="Monthly Revenue")
        fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
        fig.show()

    # Trends
    with tabs.children[1]:
        for col, title in [("revenue","Monthly Revenue"),("orders","Monthly Orders"),("rolling_3m_revenue","Rolling 3M Revenue (Avg)")]:
            fig = px.line(monthly, x="year_month", y=col, title=title)
            fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
            fig.show()

    # Customers
    with tabs.children[2]:
        top = df.groupby("customer_name")["sales_amount"].sum().sort_values(ascending=False).head(int(topn_sl.value)).reset_index()
        fig = px.bar(top, x="sales_amount", y="customer_name", orientation="h", title=f"Top {int(topn_sl.value)} Customers by Revenue")
        fig.update_layout(yaxis={"categoryorder": "total ascending"}, margin=dict(l=20, r=20, t=50, b=20))
        fig.show()

        p = pareto_curve(df, "customer_name", "sales_amount")
        pareto_fig = px.line(p, x="rank", y="cum_share", title="Customer Pareto Curve (Cumulative Revenue Share)")
        pareto_fig.update_yaxes(tickformat=".0%")
        pareto_fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
        pareto_fig.show()

        if "customer_segment" in df.columns:
            seg_df = df.groupby("customer_segment")["sales_amount"].sum().sort_values(ascending=False).reset_index()
            seg_df["share_pct"] = (seg_df["sales_amount"] / seg_df["sales_amount"].sum() * 100).round(1)
            display(seg_df)

    # Products
    with tabs.children[3]:
        top = df.groupby("product_name")["sales_amount"].sum().sort_values(ascending=False).head(int(topn_sl.value)).reset_index()
        fig = px.bar(top, x="sales_amount", y="product_name", orientation="h", title=f"Top {int(topn_sl.value)} Products by Revenue")
        fig.update_layout(yaxis={"categoryorder": "total ascending"}, margin=dict(l=20, r=20, t=50, b=20))
        fig.show()

        p = pareto_curve(df, "product_name", "sales_amount")
        pareto_fig = px.line(p, x="rank", y="cum_share", title="Product Pareto Curve (Cumulative Revenue Share)")
        pareto_fig.update_yaxes(tickformat=".0%")
        pareto_fig.update_layout(margin=dict(l=20, r=20, t=50, b=20))
        pareto_fig.show()

        if "category" in df.columns:
            cols = ["category"] + (["subcategory"] if "subcategory" in df.columns else [])
            cat_df = df.groupby(cols)["sales_amount"].sum().sort_values(ascending=False).reset_index().rename(columns={"sales_amount":"revenue"})
            display(cat_df.head(20))

    # Data Quality
    with tabs.children[4]:
        nulls = df.isna().mean().sort_values(ascending=False).head(12).reset_index()
        nulls.columns = ["field", "null_rate"]
        display(widgets.HTML("<h3>Data Quality — Selected Slice</h3>"))
        display(pd.DataFrame({"metric":["Rows"],"value":[len(df)]}))
        display(widgets.HTML("<h4>Top null rates (selected slice)</h4>"))
        display(nulls)

        display(widgets.HTML("<h4>Data quality indicators (sample extract)</h4>"))
        display(pd.DataFrame({"metric": list(dq.keys()), "value": list(dq.values())}))

for w in [start_dd, end_dd, seg_dd, cat_dd, subcat_dd, topn_sl]:
    w.observe(lambda change: render_all(), names="value")

# Interactive dashboard (render at end)
try:
    display(filters, tabs)
    render_all()
except Exception as e:
    print("Interactive widget render skipped:", e)
